<a href="https://colab.research.google.com/github/Dhananjana97/ML-Ride-Fair/blob/master/Copy_of_Copy_of_XGBoost_change_fareper_dist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import common Modules

In [38]:
import pandas as pd
import numpy as np
import xlrd


import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score

from datetime import datetime




# FileLoad

In [39]:
train_df=pd.read_csv('/content/drive/My Drive/ML Project/train.csv');
test_df=pd.read_csv('/content/drive/My Drive/ML Project/test.csv');
submission_df=pd.read_csv('/content/drive/My Drive/ML Project/sample_submission.csv')
train_df.drop_duplicates(inplace = True)
test_df.drop_duplicates(inplace = True)
train_df

# train_df = train_df[train_df['duration']!=0]
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct


# Seperate label clumn

In [40]:
lbl=[]
for index, row in train_df.iterrows():
  label=row['label']
  
  if(label=="correct"):
    lbl.append(1)
  elif(label=="incorrect"):
    lbl.append(0)
lbl

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,


In [41]:
train_df['lb']=lbl


In [42]:
train_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,lb
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct,1
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct,1
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct,1
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct,1
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct,1
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct,1
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct,1
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,correct,1


In [43]:
labels_df=pd.DataFrame(train_df['lb'])
features_df=train_df.drop(labels=['label','lb'],axis=1)
features_df

features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31


#Import model related modules

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn_pandas import CategoricalImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
# from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from pprint import pprint
from sklearn.metrics import fbeta_score
from math import radians, cos, sin, asin, sqrt 

# Calc Distance 

In [45]:


def getDistance(df):
  # The math module contains a function named 
    # radians which converts from degrees to radians. 
  dist=[]
  for index, row in df.iterrows():
    lon1 = radians(row['pick_lon']) 
    lon2 = radians(row['drop_lon']) 
    lat1 = radians(row['pick_lat']) 
    lat2 = radians(row['drop_lat']) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    distance=c * r
    dist.append(distance)
  
  return dist
      
      


In [46]:
getDistance(features_df)

[5.092769918614749,
 3.1680575840028093,
 6.305395096498956,
 0.8619461197905727,
 8.14778220061441,
 24.207039248887845,
 4.77762352223266,
 5.322543700241312,
 1.0353018356743493,
 2.930715230104055,
 14.381001132845066,
 4.515655217602484,
 9.424518630693273,
 1.4822326128545675,
 1.4400702368954967,
 13.134533272021095,
 5.123992648484041,
 1.2646648169662118,
 9.946708664968586,
 2.4989545167381277,
 2.2814676303202774,
 16.783013705293456,
 1.9297374974341521,
 4.2524115423095665,
 8.774419577321087,
 2.158639601316546,
 0.22512869093339952,
 1.5586563952189847,
 5.044182192380528,
 4.332270836375877,
 2.7020831310547613,
 2.4344507883203668,
 6.095000666312774,
 9.932622235546958,
 0.7754995733372814,
 1.1968734616119863,
 1.955358507262967,
 7.221149247721649,
 6.4991273847408655,
 2.7143833765541463,
 10.804860544656814,
 0.7983078488963857,
 1.5689911732292072,
 0.7124683760151019,
 2.4514238318216366,
 1.7361632222406627,
 0.6001069791989461,
 0.01236083468686051,
 5.5632191

In [47]:
features_df['dist']=getDistance(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077


# make bins of pickup time

In [48]:
# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
def get_time_type(df):
  pickup_slot=[]
  both="Both"
  school="School"
  office="Office"
  free="free"
  for index, row in df.iterrows():
    pickup_datetime=row['pickup_time']
    both_start="6:00"
    both_end="9:30"
    school_start="13:30"
    school_end="14:30"
    office_start="16:30"
    office_end="19:30"

    pickup_time=datetime.strptime(pickup_datetime.split(" ")[1],'%H:%M')
    both_start_time=datetime.strptime(both_start,'%H:%M')
    both_end_time=datetime.strptime(both_end,'%H:%M')
    school_start_time=datetime.strptime(school_start,'%H:%M')
    school_end_time=datetime.strptime(school_end,'%H:%M')
    office_start_time=datetime.strptime(office_start,'%H:%M')
    office_end_time=datetime.strptime(office_end,'%H:%M')
    
    if(pickup_time> both_start_time and pickup_time<both_end_time):
      pickup_slot.append(both)
      # print(pickup_time,both)
    elif(pickup_time> school_start_time and pickup_time<school_end_time):
      pickup_slot.append(school)
      # print(pickup_time,school)
    elif(pickup_time>office_start_time and pickup_time<office_end_time):
      pickup_slot.append(office)
      # print(pickup_time,office)
    else:
      pickup_slot.append(free)
      # print(pickup_time,free)
    

    
  return pickup_slot

In [49]:
features_df['Time_type']=get_time_type(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Time_type
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,free
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,free
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,free
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,free
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,free
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,free
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,free
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,free
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,free


# fare per distance

In [50]:


def get_fare_per_distance(df):
  fare_per_dist=[]
  for index, row in df.iterrows():
    if(row['dist']==0):
     
      fare_per_dist.append(0)
    else:
      fare_per_distance=(float(row['fare'])-(float(row['meter_waiting_fare'])+float(row['additional_fare'])))/float(row['dist'])
      
      fare_per_dist.append(fare_per_distance)
  
  return fare_per_dist




In [51]:
get_fare_per_distance(features_df)

[51.01742355379603,
 59.13718265287499,
 46.17315735879172,
 65.12727270428391,
 nan,
 43.562535226130485,
 53.608242425998235,
 54.55126652822712,
 82.25234136142836,
 72.81621831010278,
 47.55371296356379,
 49.84658685246301,
 46.88727534166849,
 67.9448010565946,
 80.40371714758415,
 44.0179630312083,
 55.03911560908212,
 67.061247266647,
 45.31649766595668,
 86.45215371186342,
 nan,
 40.6512210488641,
 46.10792924856668,
 52.18523602244639,
 46.796257733250904,
 92.68800585237666,
 392.7229338625503,
 57.03733053205078,
 41.42990717418455,
 38.72334079194784,
 nan,
 nan,
 49.02926453341669,
 52.54318284040338,
 690.3807795741332,
 100.16597731102988,
 49.7555817199899,
 57.25404444873436,
 47.45954060297256,
 43.805308058932596,
 50.1431737837583,
 83.24633171510443,
 61.88052658076779,
 70.17855343931808,
 42.574114947085846,
 97.8067026329749,
 65.13805263884632,
 4045.0342769448803,
 77.78654589571455,
 37.66614080892729,
 36.80712573086501,
 48.78894307550188,
 43.6946067315633

In [52]:
features_df['Fare_per_dist']=get_fare_per_distance(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Time_type,Fare_per_dist
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,free,51.017424
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,free,59.137183
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,free,46.173157
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,free,65.127273
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,free,86.605971
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,free,52.512899
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,free,62.839636
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,free,42.656035


#Fare per min

In [53]:


def get_fare_per_min(df):
  fare_per_min=[]
  for index, row in df.iterrows():
    if(row['duration']==0):
     
      fare_per_min.append(0)
    else:
      fare_per_minute=(float(row['fare'])-float(row['additional_fare']))/(float(row['duration'])/60.0)

      fare_per_min.append(fare_per_minute)
   
        
    
    
  
  return fare_per_min




In [54]:
get_fare_per_min(features_df)

[18.69208633093525,
 14.211125158027812,
 16.070285188592454,
 7.204013377926421,
 nan,
 18.570942177869092,
 12.333226324237561,
 13.843210802700677,
 14.969999999999999,
 9.644834307992202,
 1578.1615384615384,
 435.658064516129,
 13.184186971655892,
 4.795714285714285,
 20.842388059701495,
 9.415351812366739,
 12.01304347826087,
 17.91760563380282,
 11.106776180698152,
 14.548148148148147,
 nan,
 12.888853904282117,
 8.55930735930736,
 16.25294117647059,
 17.073180873180874,
 33.53296089385475,
 9.61583850931677,
 9.349514563106796,
 8.911727078891257,
 17.505391304347825,
 nan,
 nan,
 11.956171284634761,
 12.604042806183116,
 112.31958041958042,
 13.58882681564246,
 16.92,
 13.111205073995771,
 10.276844262295082,
 12.797849462365592,
 11.41812434141201,
 13.107120743034056,
 13.453579676674366,
 15.957446808510639,
 4.631369765791342,
 4.40916095890411,
 6.202424242424243,
 2.0463847203274215,
 13.995255286230012,
 9.713574660633483,
 7.205256570713392,
 16.72909090909091,
 9.7523

In [55]:
features_df['Fare_per_min']=get_fare_per_min(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Time_type,Fare_per_dist,Fare_per_min
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,free,51.017424,18.692086
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,free,59.137183,14.211125
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,free,46.173157,16.070285
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,free,65.127273,7.204013
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,free,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,free,86.605971,13.443437
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,free,52.512899,15.919944
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,free,62.839636,14.988593
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,free,42.656035,8.588112


# difference between fare_per_dist and fare_per_min

In [56]:


def get_fare_diff(df):
  fare_per_diff=[]
  for index, row in df.iterrows():
    if(row['Fare_per_dist']==0 and row['Fare_per_min']==0):
     
      fare_per_diff.append(0)
    else:
      fare_diff=(float(row['Fare_per_dist']))/(float(row['Fare_per_min']))

      fare_per_diff.append(fare_diff)
   
        
    
    
  
  return fare_per_diff




In [57]:
features_df['Fare_diff']=get_fare_diff(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Time_type,Fare_per_dist,Fare_per_min,Fare_diff
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,free,51.017424,18.692086,2.729360
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,free,59.137183,14.211125,4.161330
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,free,46.173157,16.070285,2.873201
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,free,65.127273,7.204013,9.040415
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,free,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,free,86.605971,13.443437,6.442249
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,free,52.512899,15.919944,3.298560
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,free,62.839636,14.988593,4.192497
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,free,42.656035,8.588112,4.966870


# pickup hour

In [58]:
# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
def get_pickup_hour(df):
  pickup_hour=[]
 
  for index, row in df.iterrows():
    pickup_datetime=row['pickup_time']
    

    pickup_time=pickup_datetime.split(" ")[1];
    pickup_h=pickup_time.split(":")[0]
    pickup_hour.append(pickup_h)
   
    
   
    

    
  return pickup_hour

get_pickup_hour(features_df)

['0',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '6',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '8',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',

In [59]:
features_df['pickup_hour']=get_pickup_hour(features_df)
features_df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,dist,Time_type,Fare_per_dist,Fare_per_min,Fare_diff,pickup_hour
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,5.092770,free,51.017424,18.692086,2.729360,0
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,3.168058,free,59.137183,14.211125,4.161330,0
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,6.305395,free,46.173157,16.070285,2.873201,1
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,0.861946,free,65.127273,7.204013,9.040415,2
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,8.147782,free,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,2.105376,free,86.605971,13.443437,6.442249,22
17172,213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,10.868377,free,52.512899,15.919944,3.298560,23
17173,213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,1.045518,free,62.839636,14.988593,4.192497,23
17174,213815405,10.5,858.0,115.0,0.0000,317.0,1/31/2020 23:39,1/31/2020 23:53,6.94540,79.8768,6.93574,79.9010,133.31,2.879077,free,42.656035,8.588112,4.966870,23


# fare for only trip

# Define featues

In [60]:
features=['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','dist','Fare_per_dist','Fare_per_min']
num_features=['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','fare','dist','Fare_per_dist','Fare_per_min']
# cat_features=['Time_type']
# meter_waiting_till_pickup	pickup_time	drop_time	pick_lat	pick_lon	drop_lat	drop_lon	fare	label


In [61]:
final_features_df=features_df[features]
final_features_df
# features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,dist,Fare_per_dist,Fare_per_min
0,10.5,834.0,56.0,0.0000,64.0,270.32,5.092770,51.017424,18.692086
1,10.5,791.0,47.0,0.0000,134.0,197.85,3.168058,59.137183,14.211125
2,10.5,1087.0,80.0,0.0000,61.0,301.64,6.305395,46.173157,16.070285
3,10.5,598.0,271.0,15.6638,68.0,82.30,0.861946,65.127273,7.204013
4,NaN,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,2.105376,86.605971,13.443437
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,10.868377,52.512899,15.919944
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1.045518,62.839636,14.988593
17174,10.5,858.0,115.0,0.0000,317.0,133.31,2.879077,42.656035,8.588112


In [62]:
features_df.to_csv("s.csv",index=True)

#Preprocess Pipeline

In [63]:
preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy="mean"))
])

cat_preprocessing_steps=Pipeline([
  ('cat_one_hot',OneHotEncoder())
])
preprocessor=ColumnTransformer(
    transformers = [
        ("features", preprocessing_steps, num_features)
        
    ],
    remainder="passthrough"
   
)
d=pd.DataFrame(preprocessor.fit_transform(final_features_df))
# d[0].std()
d

,0,1,2,3,4,5,6,7,8
0,-3.253376e-02,-1.070020e-01,-7.106395e-02,-6.886545e-02,-1.096375e-01,-0.069913,0.005440,-6.781763e-02,-9.414890e-03
1,-3.253376e-02,-1.122972e-01,-7.218006e-02,-6.886545e-02,4.870523e-02,-0.189631,-0.039301,-6.758590e-02,-3.860864e-02
2,-3.253376e-02,-7.584681e-02,-6.808766e-02,-6.886545e-02,-1.164237e-01,-0.018174,0.033627,-6.795589e-02,-2.649609e-02
3,-3.253376e-02,-1.360638e-01,-4.440135e-02,-3.521680e-02,-1.005894e-01,-0.380515,-0.092907,-6.741494e-02,-8.426043e-02
4,-1.511175e-17,3.349444e-18,-6.280208e-18,-1.632854e-17,1.559585e-17,0.075576,0.076454,-8.795401e-18,7.329501e-18
...,...,...,...,...,...,...,...,...,...
17167,-3.253376e-02,-1.065095e-01,-6.647550e-02,-5.721824e-02,7.657718e-01,-0.188954,-0.064003,-6.680194e-02,-4.361018e-02
17168,-3.253376e-02,5.517756e-02,-2.493145e-02,-6.886545e-02,-1.661885e-01,0.443699,0.139695,-6.777495e-02,-2.747558e-02
17169,-3.253376e-02,-1.773168e-01,-7.689252e-02,-6.886545e-02,-5.583723e-03,-0.390592,-0.088640,-6.748023e-02,-3.354339e-02
17170,-3.253376e-02,-1.040466e-01,-6.374724e-02,-6.886545e-02,4.626585e-01,-0.296249,-0.046018,-6.805627e-02,-7.524294e-02


# model define

In [64]:
# estimator=RandomForestClassifier(bootstrap= False,max_depth= 20,max_features= 'auto',min_samples_leaf= 4,min_samples_split= 2,n_estimators= 754)
estimator=XGBClassifier(learning_rate=0.0165,max_depth=47,n_estimators=271)

# estimator=XGBClassifier()
estimator
# rf_random.best_params_


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.0165, max_delta_step=0, max_depth=47,
              min_child_weight=1, missing=None, n_estimators=271, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#Full pipeline define

In [65]:
full_Pipeline=Pipeline([
  ("preprocess",preprocessor),
  ("estimator",estimator)
])
labels_df
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,dist,Fare_per_dist,Fare_per_min
0,10.5,834.0,56.0,0.0000,64.0,270.32,5.092770,51.017424,18.692086
1,10.5,791.0,47.0,0.0000,134.0,197.85,3.168058,59.137183,14.211125
2,10.5,1087.0,80.0,0.0000,61.0,301.64,6.305395,46.173157,16.070285
3,10.5,598.0,271.0,15.6638,68.0,82.30,0.861946,65.127273,7.204013
4,NaN,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,2.105376,86.605971,13.443437
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,10.868377,52.512899,15.919944
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1.045518,62.839636,14.988593
17174,10.5,858.0,115.0,0.0000,317.0,133.31,2.879077,42.656035,8.588112


In [66]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 700, num = 8)]

max_depth = [int(x) for x in np.linspace(5,80, num = 8)]
# max_depth.append(None)

learning_rate=[float(x) for x in np.linspace(0.006, 0.02, num = 5)]



# Create the random grid
random_grid = {'estimator__n_estimators': n_estimators,
               'estimator__learning_rate': learning_rate,
               'estimator__max_depth': max_depth}
pprint(random_grid)

{'estimator__learning_rate': [0.006,
                              0.0095,
                              0.013000000000000001,
                              0.0165,
                              0.02],
 'estimator__max_depth': [5, 15, 26, 37, 47, 58, 69, 80],
 'estimator__n_estimators': [100, 185, 271, 357, 442, 528, 614, 700]}


In [67]:
# from sklearn.model_selection import GridSearchCV


# rf_random = GridSearchCV (estimator=estimator,param_grid=random_grid,cv=6,scoring='f1',verbose=4,return_train_score=True)

# rf_random.get_params()

In [68]:
from sklearn.model_selection import RandomizedSearchCV


rf_random = RandomizedSearchCV(estimator = full_Pipeline, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42)

rf_random.get_params()

{'cv': 3, 'error_score': nan, 'estimator': Pipeline(memory=None,
          steps=[('preprocess',
                  ColumnTransformer(n_jobs=None, remainder='passthrough',
                                    sparse_threshold=0.3,
                                    transformer_weights=None,
                                    transformers=[('features',
                                                   Pipeline(memory=None,
                                                            steps=[('standard_scaler',
                                                                    StandardScaler(copy=True,
                                                                                   with_mean=True,
                                                                                   with_std=True)),
                                                                   ('simple_imputer',
                                                                    SimpleImputer(add_indicator=False,
    

#split dataset

In [69]:
X_train, X_eval, y_train, y_eval = train_test_split(
    final_features_df,
    labels_df,
    test_size=0.2,
    shuffle=True,
    random_state=6
)
final_features_df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,dist,Fare_per_dist,Fare_per_min
0,10.5,834.0,56.0,0.0000,64.0,270.32,5.092770,51.017424,18.692086
1,10.5,791.0,47.0,0.0000,134.0,197.85,3.168058,59.137183,14.211125
2,10.5,1087.0,80.0,0.0000,61.0,301.64,6.305395,46.173157,16.070285
3,10.5,598.0,271.0,15.6638,68.0,82.30,0.861946,65.127273,7.204013
4,NaN,NaN,NaN,NaN,NaN,358.39,8.147782,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,198.26,2.105376,86.605971,13.443437
17172,10.5,2151.0,428.0,0.0000,39.0,581.23,10.868377,52.512899,15.919944
17173,10.5,263.0,9.0,0.0000,110.0,76.20,1.045518,62.839636,14.988593
17174,10.5,858.0,115.0,0.0000,317.0,133.31,2.879077,42.656035,8.588112


# train model

In [70]:
np.testing.assert_array_equal(final_features_df.index.values,labels_df.index.values)
# payments_df


In [71]:
# rf_random.fit(X_train,y_train['lb'])
full_Pipeline.fit(X_train,y_train['lb'])


Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('features',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                           

In [72]:
# rf_random.best_params_

# full_Pipeline.score(X_eval,y_eval['lb'])

In [73]:
y_pred=full_Pipeline.predict(X_eval)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [74]:
full_Pipeline.score(X_eval,y_eval['lb'])

0.9499272197962154

In [75]:
fbeta_score(y_eval,y_pred,beta=0.5)

0.9646739130434783

In [76]:
metrics.SCORERS.keys()
features

['additional_fare',
 'duration',
 'meter_waiting',
 'meter_waiting_fare',
 'meter_waiting_till_pickup',
 'fare',
 'dist',
 'Fare_per_dist',
 'Fare_per_min']

In [77]:
from sklearn.model_selection import cross_val_score
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(full_Pipeline, final_features_df, labels_df['lb'], cv=kfold, scoring='f1')
cv_results

array([0.9678651 , 0.97735247, 0.97058824, 0.97352472, 0.97509579,
       0.97202797, 0.97956577, 0.97135582, 0.97488076, 0.97333333])

In [78]:
cv_results.mean()

0.9735589976220534

In [79]:
f1_score(y_eval, y_pred)

0.9726027397260274

# train for full dataset

In [80]:
features

['additional_fare',
 'duration',
 'meter_waiting',
 'meter_waiting_fare',
 'meter_waiting_till_pickup',
 'fare',
 'dist',
 'Fare_per_dist',
 'Fare_per_min']

In [81]:
# predict=estimator.predict(preprocessed_test_features);
full_Pipeline.fit(final_features_df,labels_df['lb'])

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('features',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                           

In [82]:
test_df['dist']=getDistance(test_df)
test_df['Time_type']=get_time_type(test_df)
test_df['Fare_per_dist']=get_fare_per_distance(test_df)
test_df['Fare_per_min']=get_fare_per_min(test_df)
test_df['Fare_diff']=get_fare_diff(test_df)
test_df['pickup_hour']=get_pickup_hour(test_df)
test_df[features]

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,dist,Fare_per_dist,Fare_per_min
0,10.5,924,42,2.44860,148,289.27,6.705702,41.206933,18.101948
1,10.5,4249,20,0.00000,91,1912.70,41.558513,45.771609,26.860908
2,10.5,1552,255,2.65880,23,394.00,5.916678,64.367402,14.826031
3,10.5,462,16,0.00000,198,154.32,3.301761,43.558571,18.677922
4,10.5,814,392,12.36920,69,147.47,2.588542,48.135517,10.096069
...,...,...,...,...,...,...,...,...,...
8571,10.5,1723,429,24.83332,3,388.48,3.934272,89.761643,13.162391
8572,10.5,1378,80,0.00000,125,379.85,7.517433,49.132466,16.082003
8573,10.5,418,56,3.28440,93,112.79,2.057225,48.125809,14.682775
8574,10.5,1604,548,31.67440,17,248.46,3.900888,52.881696,8.901247


In [83]:
# prepros_test=pd.DataFrame(preprocessor.fit_transform(test_df))
# prepros_test

In [84]:
t=full_Pipeline.predict(test_df[features])
t

array([1, 1, 1, ..., 1, 1, 1])

In [85]:
submission_df['prediction']=t

In [86]:
submission_df

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
...,...,...
8571,222856243,1
8572,222857785,1
8573,222858416,1
8574,222858691,1


In [87]:
submission_df.to_csv("submission_21.csv",index=True)